### HIV Sim Notebook - Dieter Brehm
An implementation of an HIV model

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [10]:
def make_system(gamma, mu, beta, alpha, sigma, delta, pi, tau, rho):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(R=1000, L=0, E=0, V=0)
    init /= np.sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  gamma=gamma, mu=mu, beta=beta, 
                  alpha=alpha, sigma=sigma, delta=delta,
                  pi=pi, tau=tau, rho=rho)

In [11]:
def update_func(state, t, system):
    """Update the HIV model.
    
    state: State (R, L, E, V)
    t: time
    system: System object
    
    returns: State (RLEV)
    """
    
    unpack(system)
    
    r, l, e, v = state

    #infected = system.beta * i * s    
    #recovered = system.gamma * i
    
    r_flow_in = gamma * tau
    r_flow_death = mu * r
    r_flow_out = beta * r * v
    
    l_flow_in = p * r_flow_out
    l_flow_death = mu * l
    l_flow_e = alpha * l
    
    e_flow_in = ((1 - rho) * r_flow_out + l_flow_e)
    e_flow_death = delta * e
    
    v_creation_in = pi * e
    v_flow_death = sigma * v
    
    #rDelta =  (gamma * tau) - (mu * r) - (beta * r * v)
    #lDelta = (rho * beta * r * v) - (mu * l) - (alpha * l)
    #eDelta = ((1 - rho) * beta * r * v) + (alpha * l) - (delta * e)
    #vDelta = (pi * e) - (sigma * v)
    
    #s -= infected
    #i += infected - recovered
    #r += recovered
    
    r += r_flow_in - r_flow_death - r_flow_out
    l += l_flow_in - l_flow_death - l_flow_e
    e += e_flow_in - e_flow_death
    v += v_creation_in - v_flow_death
    
    return State(R=r, L=l, E=e, V=v)